In [1]:
import regex as re
import pandas as pd
import numpy as np

!pip install pyvi
!pip install gensim
from pyvi import ViTokenizer, ViPosTagger # thư viện NLP tiếng Việt
from tqdm import tqdm
import gensim # thư viện NLP
import unicodedata

You should consider upgrading via the 'c:\users\admin\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\admin\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [2]:
# Đưa toàn bộ dữ liệu qua hàm này để chuẩn hóa lại
def covert_unicode(txt):
    if (type(txt) is not str):
        return txt
    return unicodedata.normalize('NFC', txt)

In [3]:
bang_nguyen_am = [['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a'],
                  ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
                  ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
                  ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e'],
                  ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
                  ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i'],
                  ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o'],
                  ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
                  ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
                  ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u'],
                  ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
                  ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y']]
bang_ky_tu_dau = ['', 'f', 's', 'r', 'x', 'j']

nguyen_am_to_ids = {}

for i in range(len(bang_nguyen_am)):
    for j in range(len(bang_nguyen_am[i]) - 1):
        nguyen_am_to_ids[bang_nguyen_am[i][j]] = (i, j)

def chuan_hoa_dau_tu_tieng_viet(word):
    if not is_valid_vietnam_word(word):
        return word

    chars = list(word)
    dau_cau = 0
    nguyen_am_index = []
    qu_or_gi = False
    
    # Tách dấu ra khỏi từ và lưu lại vị trí nguyên âm
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1: # Phụ âm thì bỏ qua
            continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True
                
        if y != 0: # Nếu có dấu
            dau_cau = y
            chars[index] = bang_nguyen_am[x][0]
        if not qu_or_gi or index != 1:
            nguyen_am_index.append(index)
            
    if len(nguyen_am_index) < 2:
        if qu_or_gi:
            # Trường hợp chữ "gì"
            if len(chars) == 2:
                x, y = nguyen_am_to_ids.get(chars[1])
                chars[1] = bang_nguyen_am[x][dau_cau]
            else:
                x, y = nguyen_am_to_ids.get(chars[2], (-1, -1))
                if x != -1:
                    chars[2] = bang_nguyen_am[x][dau_cau]
                else:
                    chars[1] = bang_nguyen_am[5][dau_cau] if chars[1] == 'i' else bang_nguyen_am[9][dau_cau]
            return ''.join(chars)
        return word

    for index in nguyen_am_index:
        x, y = nguyen_am_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = bang_nguyen_am[x][dau_cau]
            return ''.join(chars)

    if len(nguyen_am_index) == 2:
        if nguyen_am_index[-1] == len(chars) - 1:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            chars[nguyen_am_index[0]] = bang_nguyen_am[x][dau_cau]
        else:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    else:
        x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
    return ''.join(chars)


def is_valid_vietnam_word(word):
    # Từ hợp lệ là từ không có phụ âm bị kẹp giữa nguyên âm
    chars = list(word)
    nguyen_am_index = -1
    for index, char in enumerate(chars):
        x, _ = nguyen_am_to_ids.get(char, (-1, -1))
        if x != -1:
            if nguyen_am_index == -1:
                nguyen_am_index = index
            else:
                if (index - nguyen_am_index) != 1:
                    return False
                nguyen_am_index = index
    return True


def chuan_hoa_dau_cau_tieng_viet(sentence):
    """
    Chuyển câu tiếng việt về chuẩn gõ dấu kiểu cũ.
    :param sentence:
    :return:
    """
    if (type(sentence) is not str):
        return sentence
    
    words = sentence.split()
    for index, word in enumerate(words):
        # Tách chữ có dính dấu ngăn cách câu để chuẩn hóa
        cw = re.sub(r'(^\p{P}*)([\p{L}.]*\p{L}+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
        # print(cw)
        # Chuẩn hóa nếu chữ không có vấn đề (thường fail khi chưa tách thành công các từ)
        if len(cw) == 3:
            cw[1] = chuan_hoa_dau_tu_tieng_viet(cw[1])
            words[index] = ''.join(cw)
    return ' '.join(words)

In [5]:
total_files = 218
text_attrs = ["title","description","content","class"]

for index in range(total_files):
    df=pd.read_csv(f'scraped_data/crawling_{index}.csv')
    # Xóa dòng thiếu
    df=df[~((df['title'].isnull())|
                  (df['description'].isnull())|
                  (df['content'].isnull()))]
    for attr in text_attrs:
        df[attr]=df[attr].apply(covert_unicode)
        df[attr]=df[attr].str.lower()
        df[attr]=df[attr].apply(chuan_hoa_dau_cau_tieng_viet)
    df.to_csv(f'scraped_data/crawling_{index}.csv', index=False)
    print(f"{index} completed.")

0 completed.
1 completed.
2 completed.
3 completed.
4 completed.
5 completed.
6 completed.
7 completed.
8 completed.
9 completed.
10 completed.
11 completed.
12 completed.
13 completed.
14 completed.
15 completed.
16 completed.
17 completed.
18 completed.
19 completed.
20 completed.
21 completed.
22 completed.
23 completed.
24 completed.
25 completed.
26 completed.
27 completed.
28 completed.
29 completed.
30 completed.
31 completed.
32 completed.
33 completed.
34 completed.
35 completed.
36 completed.
37 completed.
38 completed.
39 completed.
40 completed.
41 completed.
42 completed.
43 completed.
44 completed.
45 completed.
46 completed.
47 completed.
48 completed.
49 completed.
50 completed.
51 completed.
52 completed.
53 completed.
54 completed.
55 completed.
56 completed.
57 completed.
58 completed.
59 completed.
60 completed.
61 completed.
62 completed.
63 completed.
64 completed.
65 completed.
66 completed.
67 completed.
68 completed.
69 completed.
70 completed.
71 completed.
72

In [62]:
#open("vietnamese-stopwords-dash.txt","r").read()
with open('vietnamese-stopwords-dash.txt', encoding='utf-8') as f:
    lines = f.read().splitlines()
    
lines

['a_lô',
 'a_ha',
 'ai',
 'ai_ai',
 'ai_nấy',
 'ai_đó',
 'alô',
 'amen',
 'anh',
 'anh_ấy',
 'ba',
 'ba_ba',
 'ba_bản',
 'ba_cùng',
 'ba_họ',
 'ba_ngày',
 'ba_ngôi',
 'ba_tăng',
 'bao_giờ',
 'bao_lâu',
 'bao_nhiêu',
 'bao_nả',
 'bay_biến',
 'biết',
 'biết_bao',
 'biết_bao_nhiêu',
 'biết_chắc',
 'biết_chừng_nào',
 'biết_mình',
 'biết_mấy',
 'biết_thế',
 'biết_trước',
 'biết_việc',
 'biết_đâu',
 'biết_đâu_chừng',
 'biết_đâu_đấy',
 'biết_được',
 'buổi',
 'buổi_làm',
 'buổi_mới',
 'buổi_ngày',
 'buổi_sớm',
 'bà',
 'bà_ấy',
 'bài',
 'bài_bác',
 'bài_bỏ',
 'bài_cái',
 'bác',
 'bán',
 'bán_cấp',
 'bán_dạ',
 'bán_thế',
 'bây_bẩy',
 'bây_chừ',
 'bây_giờ',
 'bây_nhiêu',
 'bèn',
 'béng',
 'bên',
 'bên_bị',
 'bên_có',
 'bên_cạnh',
 'bông',
 'bước',
 'bước_khỏi',
 'bước_tới',
 'bước_đi',
 'bạn',
 'bản',
 'bản_bộ',
 'bản_riêng',
 'bản_thân',
 'bản_ý',
 'bất_chợt',
 'bất_cứ',
 'bất_giác',
 'bất_kì',
 'bất_kể',
 'bất_kỳ',
 'bất_luận',
 'bất_ngờ',
 'bất_nhược',
 'bất_quá',
 'bất_quá_chỉ',
 'bất_thình_l

In [63]:
# Danh sách stopword
stopword = set(lines)
 
def remove_stopwords(line):
    words = []
    for word in line.strip().split():
        if word not in stopword:
            words.append(word)
    return ' '.join(words)

data_df['content']=data_df['content'].apply(remove_stopwords)

In [64]:
%time
ViTokenizer.tokenize(data_df["content"][1])

Wall time: 0 ns


'chia_sẻ lễ , thủ_tướng nguyễn xuân phúc đất đứng giai_đoạn phát_triển khát_vọng lớn . giới thành_tựu mạng 4.0 , ứng_dụng công_nghệ hiện_đại thách_thức đối quốc_gia phát_triển mong rút ngắn phát_triển . " quốc_gia khả_năng nắm_bắt thành_tựu , xu_hướng thành_công , rút ngắn phát_triển . khả_năng nắm_bắt thành_tựu đó , xét lệ chất nhân_lực . đất niềm cháu . việt nam ta thể cạnh_tranh , ganh_đua quốc_gia , năng_lực , lĩnh , khí_chất cháu " - thủ_tướng nhắn_nhủ . 2020 , tình_hình dịch covid - 19 diễn_biến phức_tạp việt nam_giới , ban tổ_chức kỳ thi olympic khu_vực quốc_tế quyết_định tổ_chức hình_thức thi trực_tuyến , quốc_gia thi quốc_gia đấy . học_sinh đánh hội thi tài bè quốc_tế , gd - đt quyết_định tổ_chức đội_tuyển tham_dự kỳ thi olympic quốc_tế phương_án thi phù_hợp an_toàn . việt nam 5 đoàn học_sinh tham_dự olympic khu_vực quốc_tế 24 lượt học_sinh tham_gia ( gồm 1 đoàn học tham_dự olympic khu_vực châu - thái_bình dương 4 đoàn tham_dự olympic quốc_tế toán , hóa học , sinh_học học ) . 

In [65]:
%%time
def tokenize(txt):
    return ViTokenizer.tokenize(txt)

data_df['content']=data_df['content'].map(tokenize)

Wall time: 2min 6s
